In [1]:
## testing printing output from console
import subprocess

cmd = [ 'echo', '"Welcome to my PySpark analysis of some StackExchange Data"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of some StackExchange Data"\n'


# Load Libraries

In [2]:
import gc #garbage collection
import time
import numpy as np
import pandas as pd
from datetime import datetime

# Load PySpark

In [3]:
%run -i '1-load-pyspark.py'

The Spark UI, version 2.4.3, is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4


# Load easyFunctions and Transformers

In [4]:
## easy functions
%run -i 'load_parquet_data.py'
%run -i 'convert_csv_to_pyarrow_parquet.py'
%run -i 'export_parquet_data.py'
%run -i 'count_total_questions.py'
%run -i 'show_save_results.py'
%run -i 'show_spark_df.py'
%run -i 'show_date_range.py'
%run -i 'trim_betw_dates.py'

## pipeline transformers
%run -i 'nltkWordPunctTokeniser.py'
%run -i 'nltkSenteniser.py'

# Choose Datasets

In [5]:
## these are the forum labels that will appear in graphs/tables etc.
data_array = [
    #'Buddhism',
    #'Economics',
    #'Fitness',
    #'Health',
    #'Interpersonal'
    'Stats',
    'English',
    #'Math',
    #'Superuser',
    #'Stackoverflow',
 
]

# Download XML Data

In [6]:
## call download script - EDIT to feed in user data_array
#!bash 0-dataset-download.sh

# Convert to Parquet

In [7]:
## for stackoverflow
#convert_csv_to_pyarrow_parquet(str_dir='initial-data/stackoverflow.stackexchange.com/')

In [8]:
## loop through data_array
#for i in data_array:
#    s = i.lower()
#    !python convert-xml-to-parquet.py "$s"

# Load Initial or Clean Data

In [9]:
%%time
print(datetime.now().time())
datasets = load_parquet_data(dataArray=data_array, kind='clean', printSchema=False)

11:09:33.031151
CPU times: user 3.24 ms, sys: 2.4 ms, total: 5.65 ms
Wall time: 3.72 s


In [10]:
## join Stack Overflow datasets
#union1 = temp1.union(temp2)
#union2 = union1.union(temp3)

In [11]:
## count questions before trimming
count_total_questions(dataArray=data_array, datasetDict=datasets)

Stats: 139302
English: 105475

Total: 244777


# Clean and Trim Data

In [12]:
%%time
print(datetime.now().time())
%run -i '2-clean-datasets.py'

10:15:06.880047

 checking columns are the right types and names 

----- Stats -----
root
 |-- title: string (nullable = true)
 |-- viewcount: long (nullable = true)
 |-- score: long (nullable = true)
 |-- clean_date: timestamp (nullable = true)
 |-- clean_body: string (nullable = true)

None
----- English -----
root
 |-- title: string (nullable = true)
 |-- viewcount: long (nullable = true)
 |-- score: long (nullable = true)
 |-- clean_date: timestamp (nullable = true)
 |-- clean_body: string (nullable = true)

None

 checking that there are no nans 

----- Stats -----
+-----+---------+-----+----------+
|title|viewcount|score|clean_body|
+-----+---------+-----+----------+
|    0|        0|    0|         0|
+-----+---------+-----+----------+

----- English -----
+-----+---------+-----+----------+
|title|viewcount|score|clean_body|
+-----+---------+-----+----------+
|    0|        0|    0|         0|
+-----+---------+-----+----------+

CPU times: user 46.5 ms, sys: 12.8 ms, total: 59.2 

In [13]:
## show range of dates for datasets before trimming
show_date_range(dataArray=data_array, datasetDict=datasets)

Stats:
2009-02-02 14:21:12.103000
2019-06-02 04:19:33.143000

English:
2009-06-16 13:06:53.033000
2019-06-02 04:02:54.070000



In [17]:
## trim data between uniform date range
datasets = trim_betw_dates(dataArray=data_array, datasetDict=datasets, dates=('2010-09-01', '2011-09-01'))

In [18]:
## show range of dates for datasets after trimming
show_date_range(dataArray=data_array, datasetDict=datasets)

Stats:
2011-09-01 00:11:37.547000
2012-08-31 21:04:31.477000

English:
2011-09-01 00:27:46.723000
2012-08-31 22:10:29.680000



In [19]:
## count questions after trimming
count_total_questions(dataArray=data_array, datasetDict=datasets)

Stats: 7134
English: 8433

Total: 15567


# Counts and LDA Feature Engineering

In [20]:
%%time
# about 23 min with 10 iterations and params
# FAILS for large datasets
print(datetime.now().time())
%run -i '3-feat-engineering.py'

###
# TRY different values of alpha and rho
# TRY different values of K
# TRY 'em' optimizer instead of 'online'

10:16:13.405301
On to Stats
On to English
On to Stats
On to English
CPU times: user 638 ms, sys: 199 ms, total: 836 ms
Wall time: 10min 14s


# Export Clean Data

In [21]:
%%time
# about 8 min
print(datetime.now().time())
export_parquet_data(dataArray=data_array, datasetDict=datasets)

10:26:28.199294
On to Stats
On to English
CPU times: user 35.1 ms, sys: 27 ms, total: 62.1 ms
Wall time: 6min 54s


# EDA

In [22]:
%%time
print(datetime.now().time())
%run -i '4-final-eda.py'

19:52:44.845519

Correlations between score and viewcount

Stats & 0.63 \\
English & 0.36 \\

Viewcount descriptives

\begin{tabular}{lrrrrr}
\toprule
0 &  count &      mean &   stddev &  min &      max \\
\midrule
Stats   &   4068 &   7589.14 &  27225.7 &   30 &   641120 \\
English &   8537 &  25154.98 &  66872.7 &   48 &  1827972 \\
\bottomrule
\end{tabular}


Score descriptives

\begin{tabular}{lrrrrr}
\toprule
0 &  count &  mean &  stddev &  min &  max \\
\midrule
Stats   &   4068 &  9.71 &   23.45 &   -3 &  925 \\
English &   8537 &  8.82 &   15.26 &   -9 &  581 \\
\bottomrule
\end{tabular}

viewcount

score



/Users/brad/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/__init__.py:424: MatplotlibDeprecationWarning: 
Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warn_deprecated("2.2", "Passing one of 'on', 'true', 'off', 'false' as a "
/Users/brad/anaconda3/lib/python3.6/site-packages/matplotlib/cbook/__init__.py:424: MatplotlibDeprecationWarning: 
Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warn_deprecated("2.2", "Passing one of 'on', 'true', 'off', 'false' as a "


\begin{tabular}{lrrl}
\toprule
{} &  score &  viewcount &                                                                     title \\
\midrule
0 &    925 &     535531 &  Making sense of principal component analysis, eigenvectors \& eigenvalues \\
0 &     -3 &        163 &                                             How to get the required data? \\
1 &     -3 &        773 &                                Distribution of a ratio of two proportions \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrl}
\toprule
{} &  score &  viewcount &                                                                title \\
\midrule
0 &    581 &      44658 &         How do you quote a passage that has used '[sic]' mistakenly? \\
0 &     -9 &        441 &  How to comprehend the phrase "put them in affluent group of people" \\
\bottomrule
\end{tabular}

CPU times: user 3.07 s, sys: 126 ms, total: 3.2 s
Wall time: 6.25 s


# Train/Test Splits

In [10]:
## garbage collector to speed up computation
collected = gc.collect()
print(f'Garbage collector: collected {collected} objects.')

Garbage collector: collected 14 objects.


In [14]:
%%time
print(datetime.now().time())
%run -i '6a-rand-train-test-split-50.py'
#%run -i '6b-time-train-test-split-50.py'

## check standard deviations of variables
print('\nStandard deviations are:\n')
for i in data_array:
    tr = round( pd.to_numeric(train[i].describe('score').select('score').toPandas().iloc[2][0]), 2 )
    te = round( pd.to_numeric(test[i].describe('score').select('score').toPandas().iloc[2][0]), 2 )
    perc = round( (te/tr - 1)*100, 2 )
    s = i.title() + ' & ' + str(tr) + ' & ' + str(te) + ' & ' + str(perc) + ' \\\\'
    print(s)

11:11:58.906233

Standard deviations are:

Stats & 15.86 & 12.2 & -23.08 \\
English & 7.6 & 7.78 & 2.37 \\
CPU times: user 40.8 ms, sys: 12.2 ms, total: 53 ms
Wall time: 2.24 s


In [15]:
## garbage collector to speed up computation
collected = gc.collect()
print(f'Garbage collector: collected {collected} objects.')

Garbage collector: collected 245 objects.


In [16]:
'''
interesting to see how skewed rus_stackoverflow posts are to more posts in recent years
'''

'\ninteresting to see how skewed rus_stackoverflow posts are to more posts in recent years\n'

# Create Results Dictionary

In [17]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

# Silly Mean Model

In [18]:
import pyspark.sql.functions as F

## choose target variable
target = 'score'

## create mean dictionaries
y_ravi_tr_means = {}

## calculate the mean of each forum, using ONLY training set
for i in data_array:
    y_ravi_tr_means[i] = train[i].select(target).rdd.flatMap(lambda x: x).mean()

## import rmse evaluator
from pyspark.ml.evaluation import RegressionEvaluator

## create dictionaries for training and testing (baseline) rmse 
base = {}
tr_rmse = {}

## modelling
for i in data_array:

    ## initial variable for timing
    t0 = time.time()
    
    ## train silly mean model by assigning training set mean for training and testing predictions
    train[i] = train[i].withColumn('mean_pred', F.lit(y_ravi_tr_means[i]))
    test[i] = test[i].withColumn('mean_pred', F.lit(y_ravi_tr_means[i]))

    ## evaluate silly mean model, on both training and testing set
    evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='mean_pred')
    tr_rmse[i] = round( evaluator.evaluate(train[i]), 2)
    base[i] = round( evaluator.evaluate(test[i]), 2)

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m mean\033[0m model is {base[i]}")

    ## record time taken
    timet = round( time.time() - t0, 2 )
    
    ## store as dictionary inside RESULTS dictionary, initiating dataset name entries first
    RESULTS[i.title()]['0silly_mean.0tr_rmse'] = tr_rmse[i]
    RESULTS[i.title()]['0silly_mean.1rmse'] = base[i]
    RESULTS[i.title()]['0silly_mean.2timet'] = timet
    
## record results
show_save_results(RESULTS)

The root-mean-square error of Stats's mean model is 12.2
The root-mean-square error of English's mean model is 7.78


,0silly_mean.0tr_rmse,0silly_mean.1rmse,0silly_mean.2timet
Stats,15.85,12.20,1.26
English,7.60,7.78,0.89


\begin{tabular}{lrrr}
\toprule
{} &  0silly\_mean.0tr\_rmse &  0silly\_mean.1rmse &  0silly\_mean.2timet \\
\midrule
Stats   &                 15.85 &              12.20 &                1.26 \\
English &                  7.60 &               7.78 &                0.89 \\
\bottomrule
\end{tabular}



In [19]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

# Viewcount Model

In [20]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['viewcount']
datet_variables = ['clean_date']

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(#maxIter=100, # this doesn't change anything
                      #regParam=0.3, # using regularisation parameter here useless since there is one feature
                      #elasticNetParam=0.8,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='viewcount_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
paramGrid = ParamGridBuilder() \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='viewcount_pred')

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    
    ## fitting on train and predicting on train/test
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
        
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m viewcount\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['1viewcount.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['1viewcount.1rmse'] = rmse
    RESULTS[i.title()]['1viewcount.2imprv'] = impr
    RESULTS[i.title()]['1viewcount.3timet'] = timet
    
## record results
show_save_results(RESULTS)

11:12:41.069154
The root-mean-square error of Stats's viewcount model is 10.12
The root-mean-square error of English's viewcount model is 7.01


,1viewcount.0tr_rmse,1viewcount.1rmse,1viewcount.2imprv,1viewcount.3timet
Stats,11.20,10.12,17.05,11.85
English,6.87,7.01,9.90,8.68


\begin{tabular}{lrrrr}
\toprule
{} &  1viewcount.0tr\_rmse &  1viewcount.1rmse &  1viewcount.2imprv &  1viewcount.3timet \\
\midrule
Stats   &                11.20 &             10.12 &              17.05 &              11.85 \\
English &                 6.87 &              7.01 &               9.90 &               8.68 \\
\bottomrule
\end{tabular}

CPU times: user 653 ms, sys: 158 ms, total: 811 ms
Wall time: 20.6 s


In [21]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [22]:
'''
Interesting that there are different improvements of viewcount over mean-only prediciton
'''

'\nInteresting that there are different improvements of viewcount over mean-only prediciton\n'

In [23]:
## garbage collector to speed up computation
collected = gc.collect()
print(f'Garbage collector: collected {collected} objects.')

Garbage collector: collected 979 objects.


## Count Model

In [24]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['body_word_cnt', 'titl_word_cnt', 'body_char_cnt', 
                   'titl_char_cnt', 'body_sent_cnt', 'titl_sent_cnt']
datet_variables = ['clean_date']

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## NUMERICAL columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create PROCESSING pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(#maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='counts_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-2, 1.]) \
    .addGrid(lr.regParam, [1e-2, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='counts_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m counts\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2counts.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2counts.1rmse'] = rmse
    RESULTS[i.title()]['2counts.2imprv'] = impr
    RESULTS[i.title()]['2counts.3timet'] = timet
    RESULTS[i.title()]['2counts.4elastic'] = ela_param
    RESULTS[i.title()]['2counts.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

11:13:01.896365
The root-mean-square error of Stats's counts model is 12.2
The root-mean-square error of English's counts model is 7.76


,2counts.0tr_rmse,2counts.1rmse,2counts.2imprv,2counts.3timet,2counts.4elastic,2counts.5regular
Stats,15.85,12.20,-0.00,12.64,1.00,1.0
English,7.57,7.76,0.26,12.78,0.01,1.0


\begin{tabular}{lrrrrrr}
\toprule
{} &  2counts.0tr\_rmse &  2counts.1rmse &  2counts.2imprv &  2counts.3timet &  2counts.4elastic &  2counts.5regular \\
\midrule
Stats   &             15.85 &          12.20 &           -0.00 &           12.64 &              1.00 &               1.0 \\
English &              7.57 &           7.76 &            0.26 &           12.78 &              0.01 &               1.0 \\
\bottomrule
\end{tabular}

CPU times: user 1.96 s, sys: 539 ms, total: 2.5 s
Wall time: 25.5 s


In [25]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

## Text Model

In [26]:
%%time
print(datetime.now().time())
# 18min 32s for 2-CV and GRIDSEARCH

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
textt_variables = ['title', 'clean_body']
datet_variables = ['clean_date']

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## textual columns
# tokenising text cols with custom transformer
nltk_tokeniser_body = nltkWordPunctTokeniser(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

nltk_tokeniser_title = nltkWordPunctTokeniser(
    inputCol='title', outputCol='titl_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

# count occurence of tokens, i.e. create dfm
cnt_vectrizr_body = CountVectorizer(inputCol='body_words', outputCol='body_raw_feats') #!!! minDF???
cnt_vectrizr_title = CountVectorizer(inputCol='titl_words', outputCol='titl_raw_feats')

# create IDF dfm
idf_body = IDF(inputCol="body_raw_feats", outputCol="body_feats")
idf_title = IDF(inputCol="titl_raw_feats", outputCol="titl_feats")

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['body_feats', 'titl_feats'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    nltk_tokeniser_body, 
    nltk_tokeniser_title,
    cnt_vectrizr_body,
    cnt_vectrizr_title,
    idf_body,
    idf_title,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(#maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='tokens_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-2, 1.]) \
    .addGrid(lr.regParam, [1e-2, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='tokens_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m tokens\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2tokens.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2tokens.1rmse'] = rmse
    RESULTS[i.title()]['2tokens.2imprv'] = impr
    RESULTS[i.title()]['2tokens.3timet'] = timet
    RESULTS[i.title()]['2tokens.4elastic'] = ela_param
    RESULTS[i.title()]['2tokens.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

11:13:27.483763
The root-mean-square error of Stats's tokens model is 13.75
The root-mean-square error of English's tokens model is 7.81


,2tokens.0tr_rmse,2tokens.1rmse,2tokens.2imprv,2tokens.3timet,2tokens.4elastic,2tokens.5regular
Stats,12.53,13.75,-12.70,444.63,1.0,1.0
English,7.29,7.81,-0.39,270.41,1.0,1.0


\begin{tabular}{lrrrrrr}
\toprule
{} &  2tokens.0tr\_rmse &  2tokens.1rmse &  2tokens.2imprv &  2tokens.3timet &  2tokens.4elastic &  2tokens.5regular \\
\midrule
Stats   &             12.53 &          13.75 &          -12.70 &          444.63 &               1.0 &               1.0 \\
English &              7.29 &           7.81 &           -0.39 &          270.41 &               1.0 &               1.0 \\
\bottomrule
\end{tabular}

CPU times: user 3.42 s, sys: 908 ms, total: 4.32 s
Wall time: 11min 55s


In [27]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

In [28]:
## check predictions aren't constant
models[data_array[0]].transform(test[data_array[0]]).select('tokens_pred').take(10)

[Row(tokens_pred=5.784398301735584),
 Row(tokens_pred=5.589520238592335),
 Row(tokens_pred=5.496234454981099),
 Row(tokens_pred=13.296713465267583),
 Row(tokens_pred=6.625309919268257),
 Row(tokens_pred=8.265681426752378),
 Row(tokens_pred=5.496234454981099),
 Row(tokens_pred=5.6869592701639595),
 Row(tokens_pred=5.496234454981099),
 Row(tokens_pred=5.496234454981099)]

In [29]:
"""why the heck does everything besides interpersonal have constant predictions - it's not the parameters or the size of the data"""
"""it's the number of questions in the data"""

"it's the number of questions in the data"

# LDA Features Model

In [30]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['btd[0]', 'btd[1]', 'btd[2]', 'btd[3]', 'btd[4]', 
                   'btd[5]', 'btd[6]', 'btd[7]', 'btd[8]', 'btd[9]', 
                   'std[0]', 'std[1]', 'std[2]', 'std[3]', 'std[4]', 
                   'std[5]', 'std[6]', 'std[7]', 'std[8]', 'std[9]',
                   'ttd[0]', 'ttd[1]', 'ttd[2]', 'ttd[3]', 'ttd[4]', 
                   'ttd[5]', 'ttd[6]', 'ttd[7]', 'ttd[8]', 'ttd[9]'] 

datet_variables = ['clean_date']

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(#maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='topics_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
    
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-2, 1.]) \
    .addGrid(lr.regParam, [1e-2, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='topics_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m topics\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2topics.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2topics.1rmse'] = rmse
    RESULTS[i.title()]['2topics.2imprv'] = impr
    RESULTS[i.title()]['2topics.3timet'] = timet
    RESULTS[i.title()]['2topics.4elastic'] = ela_param
    RESULTS[i.title()]['2topics.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

11:25:25.727229
The root-mean-square error of Stats's topics model is 12.19
The root-mean-square error of English's topics model is 7.78


,2topics.0tr_rmse,2topics.1rmse,2topics.2imprv,2topics.3timet,2topics.4elastic,2topics.5regular
Stats,15.84,12.19,0.08,9.13,1.0,1.0
English,7.60,7.78,-0.00,8.12,1.0,1.0


\begin{tabular}{lrrrrrr}
\toprule
{} &  2topics.0tr\_rmse &  2topics.1rmse &  2topics.2imprv &  2topics.3timet &  2topics.4elastic &  2topics.5regular \\
\midrule
Stats   &             15.84 &          12.19 &            0.08 &            9.13 &               1.0 &               1.0 \\
English &              7.60 &           7.78 &           -0.00 &            8.12 &               1.0 &               1.0 \\
\bottomrule
\end{tabular}

CPU times: user 1.51 s, sys: 423 ms, total: 1.93 s
Wall time: 17.3 s


In [31]:
RESULTS = {}
for i in data_array:
    # capitalise keysa
    RESULTS[i.title()] = {}

# Final Model

In [32]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['btd[0]', 'btd[1]', 'btd[2]', 'btd[3]', 'btd[4]', 
                   'btd[5]', 'btd[6]', 'btd[7]', 'btd[8]', 'btd[9]', 
                   'std[0]', 'std[1]', 'std[2]', 'std[3]', 'std[4]', 
                   'std[5]', 'std[6]', 'std[7]', 'std[8]', 'std[9]',
                   'ttd[0]', 'ttd[1]', 'ttd[2]', 'ttd[3]', 'ttd[4]', 
                   'ttd[5]', 'ttd[6]', 'ttd[7]', 'ttd[8]', 'ttd[9]',
                   'body_word_cnt', 'titl_word_cnt', 'body_char_cnt', 
                   'titl_char_cnt', 'body_sent_cnt', 'titl_sent_cnt']
datet_variables = ['clean_date']

### DATE columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

### NUMERICAL columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])


## create PROCESSING pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    numic_pipeline,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(#maxIter=100,
                      #regParam=1,
                      #elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='finalm_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
    
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-2, 1.]) \
    .addGrid(lr.regParam, [1e-2, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='finalm_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m final\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['3final.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['3final.1rmse'] = rmse
    RESULTS[i.title()]['3final.2imprv'] = impr
    RESULTS[i.title()]['3final.3timet'] = timet
    RESULTS[i.title()]['3final.4elastic'] = ela_param
    RESULTS[i.title()]['3final.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

11:25:43.087606
The root-mean-square error of Stats's final model is 12.19
The root-mean-square error of English's final model is 7.78


,3final.0tr_rmse,3final.1rmse,3final.2imprv,3final.3timet,3final.4elastic,3final.5regular
Stats,15.84,12.19,0.08,9.16,1.0,1.0
English,7.60,7.78,-0.00,8.27,1.0,1.0


\begin{tabular}{lrrrrrr}
\toprule
{} &  3final.0tr\_rmse &  3final.1rmse &  3final.2imprv &  3final.3timet &  3final.4elastic &  3final.5regular \\
\midrule
Stats   &            15.84 &         12.19 &           0.08 &           9.16 &              1.0 &              1.0 \\
English &             7.60 &          7.78 &          -0.00 &           8.27 &              1.0 &              1.0 \\
\bottomrule
\end{tabular}

CPU times: user 1.56 s, sys: 435 ms, total: 2 s
Wall time: 17.5 s


In [33]:
## check predictions aren't constant
models[data_array[0]].transform(test[data_array[0]]).select('finalm_pred').take(10)

[Row(finalm_pred=6.361150704095528),
 Row(finalm_pred=6.486751918831093),
 Row(finalm_pred=6.5020183317054325),
 Row(finalm_pred=6.332715831420088),
 Row(finalm_pred=6.396022661404337),
 Row(finalm_pred=6.504573459336636),
 Row(finalm_pred=6.378045727212639),
 Row(finalm_pred=6.345291775885759),
 Row(finalm_pred=6.322897788846911),
 Row(finalm_pred=6.416152443848872)]

In [ ]:
fin

# Save predictions

In [ ]:
(trained_pipeline
 .transform(datasets['english'])
 .select(
    indep_text_variables + ["prediction"]
 )
 .write
 .parquet("linreg_prediction.parquet")
)

In [ ]:
linreg_predictions = spark.read.parquet("linreg_prediction.parquet")

In [ ]:
linreg_predictions.toPandas().head()

In [ ]:
linreg_predictions.select("prediction").describe().toPandas()

# Save pipelines

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

reloaded = load("pipeline.joblib")

#Now we can predict directly!

reloaded.predict(X)[:10]

In [ ]:
## save models DOESN'T WORK BECAUSE: 'NLTKWordPunctTokenizer' object has no attribute '_to_java'
for i in data_array:
    param_dict[i].save(f'{i}-pipeline') 

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook-pipelines.ipynb